open the following link with a new tab

<a href="https://colab.research.google.com/github/nzhinusoftcm/review-on-collaborative-filtering/blob/master/7.Explainable_Matrix_Factorization.ipynb" target="_blank">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Explainable Matrix Factorization (EMF)

### How to quantify explainability ?

- Use the rating distribution within the active user’s neighborhood. 
- If many neighbors have rated the recommended item, then this can provide a basis upon which to explain the recommendations, using neighborhood style explanation mechanisms

According to [(Abdollahi and Nasraoui, 2016)](https://www.researchgate.net/publication/301616080_Explainable_Matrix_Factorization_for_Collaborative_Filtering), an item $i$ is consider to be explainable for user $u$ if a considerable number of its neighbors rated item $i$. The explainability score $E_{ui}$ is the percentage of user $u$'s neighbors who have rated item $i$.

\begin{equation}
E_{ui} = \frac{|N_k^{(i)}(u)|}{|N_k(u)|},
\end{equation}

where $N_k(u)$ is the set of $k$ nearest neighbors of user $u$ and $N_k^{(i)}(u)$ is the set of user $u$'s neighbors who have rated item $i$. However, only explainable scores above an optimal threshold $\theta$ are accepted.

\begin{equation}
W_{ui} = \begin{cases} E_{ui} \text{  } if \text{  } E_{ui} > \theta \\ 0 \text{ } otherwise \end{cases},
\end{equation}

By including explainability weight in the training algorithm, the new objective function, to be minimized over the set of known ratings, has been formulated by [(Abdollahi and Nasraoui, 2016)](https://www.researchgate.net/publication/301616080_Explainable_Matrix_Factorization_for_Collaborative_Filtering) as:

\begin{equation}
 J = \sum_{(u,i)\in \kappa} (R_{ui} - \hat{R}_{ui})^2 +\frac{\beta}{2}(||P_u||^2 + ||Q_i||^2) + \frac{\lambda}{2}(P_u-Q_i)^2W_{ui},
\end{equation}

here, $\frac{\beta}{2}(||P_u||^2 + ||Q_i||^2)$ is the $L_2$ regularization term weighted by the coefficient $\beta$, and $\lambda$ is an explainability regularization coefficient that controls the smoothness of the new representation and tradeoff between explainability and accuracy. The idea here is that if item $i$ is explainable for user $u$, then their representations in the latent space, $Q_i$ and $P_u$, should be close to each other. Stochastic Gradient descent can be used to optimize the objectve function.

\begin{equation}
P_u \leftarrow P_u + \alpha\left(2(R_{u,i}-P_uQ_i^{\top})Q_i - \beta P_u - \lambda(P_u-Q_i)W_{ui}\right)
\end{equation}
\begin{equation}
Q_i \leftarrow Q_i + \alpha\left(2(R_{u,i}-P_uQ_i^{\top})P_u - \beta Q_i + \lambda(P_u-Q_i)W_{ui}\right)
\end{equation}

## Explainable Matrix Factorization : Model definition

In [ ]:
import os

if not (os.path.exists("recsys.zip") or os.path.exists("recsys")):
    !wget https://github.com/nzhinusoftcm/review-on-collaborative-filtering/raw/master/recsys.zip    
    !unzip recsys.zip

### requirements

```
matplotlib==3.2.2
numpy==1.18.1
pandas==1.0.5
python==3.6.10
scikit-learn==0.23.1
scipy==1.5.0
```

In [1]:
from recsys.memories.UserToUser import UserToUser

from recsys.preprocessing import mean_ratings
from recsys.preprocessing import normalized_ratings
from recsys.preprocessing import  encode_data
from recsys.preprocessing import split_data
from recsys.preprocessing import rating_matrix
from recsys.preprocessing import get_examples

from recsys.datasets import ml100k, ml1m, mlLastedSmall

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import os

### Compute Explainable Scores

Explainable score are computed using neighborhood based similarities. Here, we are using the user based algorithme to compute similarities

In [2]:
def explainable_score(user2user, users, items, theta=0):
        
    # initialize explainable score to zeros
    W = np.zeros((len(users), len(items)))

    for u in range(len(users)):            
        user_neighbors = user2user.neighbors[u][1:]
        candidate_items = user2user.find_user_candidate_items(u,user_neighbors)
        candidate_items = iencoder.transform(candidate_items)

        for i in candidate_items:                
            user_who_rated_i, similar_user_who_rated_i = \
                user2user.similar_users_who_rated_this_item(i, user_neighbors)

            if len(user_who_rated_i) == 0:
                w = 0.0
            else:
                w = len(similar_user_who_rated_i) / len(user_who_rated_i)

            W[u,i] =  w  if w > theta else 0.0 

    return W

### Explainable Matrix Factorization Model

In [3]:
class ExplainableMatrixFactorization:
    
    def __init__(self, m, n, W, alpha=0.001, beta=0.01, lamb=0.1, k=10):
        """
            - R : Rating matrix of shape (m,n) 
            - W : Explainability Weights of shape (m,n)
            - k : number of latent factors
            - beta : L2 regularization parameter
            - lamb : explainability regularization coefficient
            - theta : threshold above which an item is explainable for a user
        """
        self.W = W
        self.m = m
        self.n = n
        
        np.random.seed(64)
        
        # initialize the latent factor matrices P and Q (of shapes (m,k) and (n,k) respectively) that will be learnt
        self.k = k
        self.P = np.random.normal(size=(self.m,k))
        self.Q = np.random.normal(size=(self.n,k))
        
        # hyperparameter initialization
        self.alpha = alpha
        self.beta = beta
        self.lamb = lamb
        
        # training history
        self.history = {
            "epochs":[],
            "loss":[],
            "val_loss":[],
        }
        
    def print_training_parameters(self):
        print('Training EMF')
        print(f'k={self.k} \t alpha={self.alpha} \t beta={self.beta} \t lambda={self.lamb}')
        
    def update_rule(self, u, i, error):
        self.P[u] = self.P[u] + \
            self.alpha * ( 2 * error * self.Q[i] - self.beta * self.P[u] - self.lamb * ( self.P[u] - self.Q[i] ) * self.W[u,i])
        
        self.Q[i] = self.Q[i] + \
            self.alpha * (2 * error * self.P[u] - self.beta * self.Q[i] + self.lamb * ( self.P[u] - self.Q[i] ) * self.W[u,i])
        
    def mae(self,  x_train, y_train):
        """
        returns the Mean Absolute Error
        """
        # number of training exemples
        M = x_train.shape[0]
        error = 0
        for pair, r in zip(x_train, y_train):
            u, i = pair
            error += np.absolute(r - np.dot(self.P[u], self.Q[i]))
        return error/M
    
    def print_training_progress(self, epoch, epochs, error, val_error, steps=5):
        if epoch == 1 or epoch % steps == 0 :
                print("epoch {}/{} - loss : {} - val_loss : {}".format(epoch, epochs, round(error,3), round(val_error,3)))
                
    def learning_rate_schedule(self, epoch, target_epochs = 20):
        if (epoch >= target_epochs) and (epoch % target_epochs == 0):
                factor = epoch // target_epochs
                self.alpha = self.alpha * (1 / (factor * 20))
                print("\nLearning Rate : {}\n".format(self.alpha))
        
    def fit(self, x_train, y_train, validation_data, epochs=10):
        """
        Train latent factors P and Q according to the training set
        
        :param
            - x_train : training pairs (u,i) for which rating r_ui is known
            - y_train : set of ratings r_ui for all training pairs (u,i)
            - validation_data : tuple (x_test, y_test)
            - epochs : number of time to loop over the entire training set. 
            10 epochs by default
            
        Note that u and i are encoded values of userid and itemid
        """
        self.print_training_parameters()
        
        # get validation data
        x_test, y_test = validation_data
        
        for epoch in range(1, epochs+1):
            for pair, r in zip(x_train, y_train):                
                u,i = pair                
                r_hat = np.dot(self.P[u], self.Q[i])                
                e = r - r_hat
                self.update_rule(u, i, error=e)
                
            # training and validation error  after this epochs
            error = self.mae(x_train, y_train)
            val_error = self.mae(x_test, y_test)
            self.update_history(epoch, error, val_error)            
            self.print_training_progress(epoch, epochs, error, val_error, steps=1)
        
        return self.history
    
    def update_history(self, epoch, error, val_error):
        self.history['epochs'].append(epoch)
        self.history['loss'].append(error)
        self.history['val_loss'].append(val_error)
    
    def evaluate(self, x_test, y_test):
        """
        compute the global error on the test set
        
        :param
            - x_test : test pairs (u,i) for which rating r_ui is known
            - y_test : set of ratings r_ui for all test pairs (u,i)
        """
        error = self.mae(x_test, y_test)
        print(f"validation error : {round(error,3)}")
      
    def predict(self, userid, itemid):
        """
        Make rating prediction for a user on an item

        :param
        - userid
        - itemid

        :return
        - r : predicted rating
        """
        # encode user and item ids to be able to access their latent factors in
        # matrices P and Q
        u = uencoder.transform([userid])[0]
        i = iencoder.transform([itemid])[0]

        # rating prediction using encoded ids. Dot product between P_u and Q_i
        r = np.dot(self.P[u], self.Q[i])

        return r

    def recommend(self, userid, N=30):
        """
        make to N recommendations for a given user

        :return 
        - (top_items,preds) : top N items with the highest predictions 
        """
        # encode the userid
        u = uencoder.transform([userid])[0]

        # predictions for this user on all product
        predictions = np.dot(self.P[u], self.Q.T)

        # get the indices of the top N predictions
        top_idx = np.flip(np.argsort(predictions))[:N]

        # decode indices to get their corresponding itemids
        top_items = iencoder.inverse_transform(top_idx)

        # take corresponding predictions for top N indices
        preds = predictions[top_idx]

        return top_items, preds

In [4]:
epochs = 30

# Model Evaluation

## 1. MovieLens Lasted Small dataset

### 1.1. Evaluation on raw data

In [5]:
# load data
ratings, movies = mlLastedSmall.load()

users = sorted(ratings['userid'].unique())
items = sorted(ratings['itemid'].unique())

m = len(users)
n = len(items)

# create the user to user model for similarity measure
usertouser = UserToUser(ratings, movies)

# get examples as tuples of userids and itemids and labels from normalize ratings
raw_examples, raw_labels = get_examples(ratings)

# train test split
(train_examples, test_examples), (train_labels, test_labels) = split_data(examples=raw_examples, labels=raw_labels)

examples = (train_examples, test_examples)
labels = (train_labels, test_labels)

# encode train and test examples
(X_train, X_test), (y_train, y_test), (uencoder, iencoder) = encode_data( ratings, examples = examples, labels = labels)

Normalize users ratings ...
Create the similarity model ...
Compute nearest neighbors ...
User to user recommendation model created with success ...


In [6]:
# compute explainable score
W = explainable_score(usertouser, users, items)

In [7]:
# initialize the model
EMF = ExplainableMatrixFactorization(m, n, W, alpha=0.01, beta=0.4, lamb=0.01, k=10)

history = EMF.fit(X_train, y_train, epochs=epochs, validation_data=(X_test, y_test))

Training EMF
k=10 	 alpha=0.01 	 beta=0.4 	 lambda=0.01
epoch 1/30 - loss : 1.617 - val_loss : 1.771
epoch 2/30 - loss : 1.103 - val_loss : 1.268
epoch 3/30 - loss : 0.941 - val_loss : 1.109
epoch 4/30 - loss : 0.86 - val_loss : 1.032
epoch 5/30 - loss : 0.81 - val_loss : 0.987
epoch 6/30 - loss : 0.776 - val_loss : 0.957
epoch 7/30 - loss : 0.752 - val_loss : 0.936
epoch 8/30 - loss : 0.733 - val_loss : 0.921
epoch 9/30 - loss : 0.718 - val_loss : 0.908
epoch 10/30 - loss : 0.706 - val_loss : 0.898
epoch 11/30 - loss : 0.696 - val_loss : 0.89
epoch 12/30 - loss : 0.687 - val_loss : 0.882
epoch 13/30 - loss : 0.68 - val_loss : 0.877
epoch 14/30 - loss : 0.674 - val_loss : 0.872
epoch 15/30 - loss : 0.668 - val_loss : 0.867
epoch 16/30 - loss : 0.663 - val_loss : 0.863
epoch 17/30 - loss : 0.659 - val_loss : 0.86
epoch 18/30 - loss : 0.655 - val_loss : 0.857
epoch 19/30 - loss : 0.651 - val_loss : 0.854
epoch 20/30 - loss : 0.648 - val_loss : 0.852
epoch 21/30 - loss : 0.645 - val_loss 

In [8]:
EMF.evaluate(X_test, y_test)

validation error : 0.838


### 1.2. Evaluation on normalized data

In [9]:
# load data
ratings, movies = mlLastedSmall.load()

# create the user to user model for similarity measure
# usertouser = UserToUser(ratings, movies)

# normalize ratings by substracting means
normalized_column_name = "norm_rating"
ratings = normalized_ratings(ratings, norm_column=normalized_column_name)

# get examples as tuples of userids and itemids and labels from normalize ratings
raw_examples, raw_labels = get_examples(ratings, labels_column=normalized_column_name)

# train test split
(train_examples, test_examples), (train_labels, test_labels) = split_data(examples=raw_examples, labels=raw_labels)

examples = (train_examples, test_examples)
labels = (train_labels, test_labels)

# encode train and test examples
(X_train, X_test), (y_train, y_test), (uencoder, iencoder) = encode_data( ratings, examples = examples, labels = labels)

In [10]:
# initialize the model
EMF = ExplainableMatrixFactorization(m, n, W, alpha=0.022, beta=0.4, lamb=0.01, k=10)

history = EMF.fit(X_train, y_train, epochs=epochs, validation_data=(X_test, y_test))

Training EMF
k=10 	 alpha=0.022 	 beta=0.4 	 lambda=0.01
epoch 1/30 - loss : 0.739 - val_loss : 0.796
epoch 2/30 - loss : 0.721 - val_loss : 0.773
epoch 3/30 - loss : 0.715 - val_loss : 0.764
epoch 4/30 - loss : 0.711 - val_loss : 0.759
epoch 5/30 - loss : 0.707 - val_loss : 0.754
epoch 6/30 - loss : 0.701 - val_loss : 0.75
epoch 7/30 - loss : 0.693 - val_loss : 0.744
epoch 8/30 - loss : 0.685 - val_loss : 0.738
epoch 9/30 - loss : 0.677 - val_loss : 0.731
epoch 10/30 - loss : 0.67 - val_loss : 0.726
epoch 11/30 - loss : 0.663 - val_loss : 0.721
epoch 12/30 - loss : 0.658 - val_loss : 0.717
epoch 13/30 - loss : 0.652 - val_loss : 0.714
epoch 14/30 - loss : 0.648 - val_loss : 0.711
epoch 15/30 - loss : 0.644 - val_loss : 0.708
epoch 16/30 - loss : 0.64 - val_loss : 0.706
epoch 17/30 - loss : 0.636 - val_loss : 0.704
epoch 18/30 - loss : 0.633 - val_loss : 0.702
epoch 19/30 - loss : 0.63 - val_loss : 0.701
epoch 20/30 - loss : 0.627 - val_loss : 0.699
epoch 21/30 - loss : 0.624 - val_los

## 2. MovieLens 100k

### 2.1. Evaluation on raw data

In [23]:
# load data
ratings, movies = ml100k.load()

users = sorted(ratings['userid'].unique())
items = sorted(ratings['itemid'].unique())

m = len(users)
n = len(items)

# create the user to user model for similarity measure
usertouser = UserToUser(ratings, movies)

# get examples as tuples of userids and itemids and labels from normalize ratings
raw_examples, raw_labels = get_examples(ratings)

# train test split
(train_examples, test_examples), (train_labels, test_labels) = split_data(examples=raw_examples, labels=raw_labels)

examples = (train_examples, test_examples)
labels = (train_labels, test_labels)

# encode train and test examples
(X_train, X_test), (y_train, y_test), (uencoder, iencoder) = encode_data( ratings, examples = examples, labels = labels)

Normalize users ratings ...
Create the similarity model ...
Compute nearest neighbors ...
User to user recommendation model created with success ...


In [24]:
# compute explainable score
W = explainable_score(usertouser, users, items)

In [28]:
# initialize the model
EMF = ExplainableMatrixFactorization(m, n, W, alpha=0.01, beta=0.4, lamb=0.01, k=10)

history = EMF.fit(X_train, y_train, epochs=epochs, validation_data=(X_test, y_test))

Training EMF
k=10 	 alpha=0.01 	 beta=0.4 	 lambda=0.01
epoch 1/30 - loss : 0.919 - val_loss : 1.021
epoch 2/30 - loss : 0.79 - val_loss : 0.868
epoch 3/30 - loss : 0.766 - val_loss : 0.832
epoch 4/30 - loss : 0.757 - val_loss : 0.817
epoch 5/30 - loss : 0.753 - val_loss : 0.808
epoch 6/30 - loss : 0.751 - val_loss : 0.803
epoch 7/30 - loss : 0.749 - val_loss : 0.799
epoch 8/30 - loss : 0.748 - val_loss : 0.795
epoch 9/30 - loss : 0.746 - val_loss : 0.793
epoch 10/30 - loss : 0.745 - val_loss : 0.79
epoch 11/30 - loss : 0.743 - val_loss : 0.788
epoch 12/30 - loss : 0.742 - val_loss : 0.786
epoch 13/30 - loss : 0.74 - val_loss : 0.784
epoch 14/30 - loss : 0.739 - val_loss : 0.783
epoch 15/30 - loss : 0.738 - val_loss : 0.781
epoch 16/30 - loss : 0.737 - val_loss : 0.78
epoch 17/30 - loss : 0.736 - val_loss : 0.779
epoch 18/30 - loss : 0.735 - val_loss : 0.778
epoch 19/30 - loss : 0.735 - val_loss : 0.777
epoch 20/30 - loss : 0.734 - val_loss : 0.777
epoch 21/30 - loss : 0.733 - val_loss

### 2.2. Evaluation on normalized data

In [29]:
# load data
ratings, movies = ml100k.load()

# create the user to user model for similarity measure
# usertouser = UserToUser(ratings, movies)

# normalize ratings by substracting means
normalized_column_name = "norm_rating"
ratings = normalized_ratings(ratings, norm_column=normalized_column_name)

# get examples as tuples of userids and itemids and labels from normalize ratings
raw_examples, raw_labels = get_examples(ratings, labels_column=normalized_column_name)

# train test split
(train_examples, test_examples), (train_labels, test_labels) = split_data(examples=raw_examples, labels=raw_labels)

examples = (train_examples, test_examples)
labels = (train_labels, test_labels)

# encode train and test examples
(X_train, X_test), (y_train, y_test), (uencoder, iencoder) = encode_data( ratings, examples = examples, labels = labels)

In [30]:
# initialize the model
EMF = ExplainableMatrixFactorization(m, n, W, alpha=0.02, beta=0.4, lamb=0.01, k=10)

history = EMF.fit(X_train, y_train, epochs=epochs, validation_data=(X_test, y_test))

Training EMF
k=10 	 alpha=0.02 	 beta=0.4 	 lambda=0.01
epoch 1/30 - loss : 0.805 - val_loss : 0.852
epoch 2/30 - loss : 0.788 - val_loss : 0.825
epoch 3/30 - loss : 0.765 - val_loss : 0.798
epoch 4/30 - loss : 0.747 - val_loss : 0.778
epoch 5/30 - loss : 0.738 - val_loss : 0.767
epoch 6/30 - loss : 0.733 - val_loss : 0.761
epoch 7/30 - loss : 0.73 - val_loss : 0.757
epoch 8/30 - loss : 0.728 - val_loss : 0.755
epoch 9/30 - loss : 0.726 - val_loss : 0.753
epoch 10/30 - loss : 0.725 - val_loss : 0.751
epoch 11/30 - loss : 0.723 - val_loss : 0.75
epoch 12/30 - loss : 0.722 - val_loss : 0.749
epoch 13/30 - loss : 0.721 - val_loss : 0.748
epoch 14/30 - loss : 0.72 - val_loss : 0.747
epoch 15/30 - loss : 0.719 - val_loss : 0.746
epoch 16/30 - loss : 0.718 - val_loss : 0.745
epoch 17/30 - loss : 0.718 - val_loss : 0.745
epoch 18/30 - loss : 0.717 - val_loss : 0.744
epoch 19/30 - loss : 0.716 - val_loss : 0.744
epoch 20/30 - loss : 0.716 - val_loss : 0.744
epoch 21/30 - loss : 0.715 - val_los

## 3. MovieLens 1M

### 3.1. Evaluation on raw data

In [31]:
# load data
ratings, movies = ml1m.load()

users = sorted(ratings['userid'].unique())
items = sorted(ratings['itemid'].unique())

m = len(users)
n = len(items)

# create the user to user model for similarity measure
usertouser = UserToUser(ratings, movies)

# get examples as tuples of userids and itemids and labels from normalize ratings
raw_examples, raw_labels = get_examples(ratings)

# train test split
(train_examples, test_examples), (train_labels, test_labels) = split_data(examples=raw_examples, labels=raw_labels)

examples = (train_examples, test_examples)
labels = (train_labels, test_labels)

# encode train and test examples
(X_train, X_test), (y_train, y_test), (uencoder, iencoder) = encode_data( ratings, examples = examples, labels = labels)

Normalize users ratings ...
Create the similarity model ...
Compute nearest neighbors ...
User to user recommendation model created with success ...


In [32]:
# compute explainable score
W = explainable_score(usertouser, users, items)

In [33]:
# initialize the model
EMF = ExplainableMatrixFactorization(m, n, W, alpha=0.01, beta=0.4, lamb=0.01, k=10)

history = EMF.fit(X_train, y_train, epochs=epochs, validation_data=(X_test, y_test))

Training EMF
k=10 	 alpha=0.01 	 beta=0.4 	 lambda=0.01
epoch 1/30 - loss : 0.782 - val_loss : 0.81
epoch 2/30 - loss : 0.763 - val_loss : 0.783
epoch 3/30 - loss : 0.76 - val_loss : 0.777
epoch 4/30 - loss : 0.759 - val_loss : 0.774
epoch 5/30 - loss : 0.757 - val_loss : 0.772
epoch 6/30 - loss : 0.756 - val_loss : 0.769
epoch 7/30 - loss : 0.754 - val_loss : 0.767
epoch 8/30 - loss : 0.752 - val_loss : 0.766
epoch 9/30 - loss : 0.751 - val_loss : 0.764
epoch 10/30 - loss : 0.75 - val_loss : 0.763
epoch 11/30 - loss : 0.749 - val_loss : 0.762
epoch 12/30 - loss : 0.749 - val_loss : 0.761
epoch 13/30 - loss : 0.748 - val_loss : 0.761
epoch 14/30 - loss : 0.748 - val_loss : 0.76
epoch 15/30 - loss : 0.747 - val_loss : 0.76
epoch 16/30 - loss : 0.747 - val_loss : 0.76
epoch 17/30 - loss : 0.747 - val_loss : 0.759
epoch 18/30 - loss : 0.747 - val_loss : 0.759
epoch 19/30 - loss : 0.746 - val_loss : 0.759
epoch 20/30 - loss : 0.746 - val_loss : 0.759
epoch 21/30 - loss : 0.746 - val_loss :

### 3.2. Evaluation on normalized data

In [34]:
# load data
ratings, movies = ml1m.load()

# create the user to user model for similarity measure
# usertouser = UserToUser(ratings, movies)

# normalize ratings by substracting means
normalized_column_name = "norm_rating"
ratings = normalized_ratings(ratings, norm_column=normalized_column_name)

# get examples as tuples of userids and itemids and labels from normalize ratings
raw_examples, raw_labels = get_examples(ratings, labels_column=normalized_column_name)

# train test split
(train_examples, test_examples), (train_labels, test_labels) = split_data(examples=raw_examples, labels=raw_labels)

examples = (train_examples, test_examples)
labels = (train_labels, test_labels)

# encode train and test examples
(X_train, X_test), (y_train, y_test), (uencoder, iencoder) = encode_data( ratings, examples = examples, labels = labels)

In [35]:
# initialize the model
EMF = ExplainableMatrixFactorization(m, n, W, alpha=0.01, beta=0.4, lamb=0.01, k=10)

history = EMF.fit(X_train, y_train, epochs=epochs, validation_data=(X_test, y_test))

Training EMF
k=10 	 alpha=0.01 	 beta=0.4 	 lambda=0.01
epoch 1/30 - loss : 0.821 - val_loss : 0.839
epoch 2/30 - loss : 0.796 - val_loss : 0.811
epoch 3/30 - loss : 0.757 - val_loss : 0.77
epoch 4/30 - loss : 0.741 - val_loss : 0.754
epoch 5/30 - loss : 0.736 - val_loss : 0.747
epoch 6/30 - loss : 0.733 - val_loss : 0.744
epoch 7/30 - loss : 0.732 - val_loss : 0.742
epoch 8/30 - loss : 0.731 - val_loss : 0.741
epoch 9/30 - loss : 0.731 - val_loss : 0.741
epoch 10/30 - loss : 0.731 - val_loss : 0.74
epoch 11/30 - loss : 0.731 - val_loss : 0.74
epoch 12/30 - loss : 0.731 - val_loss : 0.74
epoch 13/30 - loss : 0.731 - val_loss : 0.74
epoch 14/30 - loss : 0.73 - val_loss : 0.739
epoch 15/30 - loss : 0.73 - val_loss : 0.739
epoch 16/30 - loss : 0.73 - val_loss : 0.739
epoch 17/30 - loss : 0.73 - val_loss : 0.739
epoch 18/30 - loss : 0.73 - val_loss : 0.739
epoch 19/30 - loss : 0.73 - val_loss : 0.739
epoch 20/30 - loss : 0.73 - val_loss : 0.739
epoch 21/30 - loss : 0.73 - val_loss : 0.739


# Ratings prediction

In [36]:
# get list of top N items with their corresponding predicted ratings
userid = 42
recommended_items, predictions = EMF.recommend(userid=userid)

# find corresponding movie titles
top_N = list(zip(recommended_items,predictions))
top_N = pd.DataFrame(top_N, columns=['itemid','predictions'])
top_N.predictions = top_N.predictions + ratings.loc[ratings.userid==userid].rating_mean.values[0]
List = pd.merge(top_N, movies, on='itemid', how='inner')

# show the list
List

,itemid,predictions,title,genres
0,3460,4.400590,Hillbillys in a Haunted House (1967),Comedy
1,1316,4.318431,Anna (1996),Drama
2,2258,4.180132,Master Ninja I (1984),Action
3,545,4.137391,Harlem (1993),Drama
4,789,4.127738,"I, Worst of All (Yo, la peor de todas) (1990)",Drama
5,701,4.120999,Daens (1992),Drama
6,642,4.107454,Roula (1995),Drama
7,3305,4.102270,Bluebeard (1944),Film-Noir|Horror
8,790,4.096415,An Unforgettable Summer (1994),Drama
9,787,4.074466,"Gate of Heavenly Peace, The (1995)",Documentary


**Note**: The recommendation list may content items already purchased by the user. This is just an illustration of how to implement matrix factorization recommender system. You can optimize the recommended list and return the top rated items that the user has not already purchased.

## Reference

1. Yehuda Koren et al. (2009). <a href='https://ieeexplore.ieee.org/document/5197422'>Matrix Factorization Techniques for Recommender Systems</a>
2. Abdollahi and Nasraoui (2016). [Explainable Matrix Factorization for Collaborative Filtering](https://www.researchgate.net/publication/301616080_Explainable_Matrix_Factorization_for_Collaborative_Filtering)
3. Abdollahi and Nasraoui (2017). [Using Explainability for Constrained Matrix Factorization](https://dl.acm.org/doi/abs/10.1145/3109859.3109913)
4. Shuo Wang et al, (2018). [Explainable Matrix Factorization with Constraints on Neighborhood in the Latent Space](https://dl.acm.org/doi/abs/10.1145/3109859.3109913)

## Author

<a href="https://www.linkedin.com/in/carmel-wenga-871876178/">Carmel WENGA</a>, Applied Machine Learning Research Engineer | <a href="https://shoppinglist.cm/fr/">ShoppingList</a>, Nzhinusoft